Preprocessing and Preparing Data
=========

Topics:
 - Splitting your data into training and testing (also k-folds) ???
 - **Principle Component Analysis**
 - **Non-negative Matrix Factorization (NMF)** ???
 - **t-SNE** ???

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
import numpy as np

Test-train splitting
-------

Scikit-learn has an option:

In [ ]:
from sklearn.model_selection import train_test_split

def split() :
    x_train, x_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)

    # Another option is *stratify*, which allows you to balance your samples to make sure there are
    # enough of a certain y output class in each sample.

Normalizing Data
--------

In [ ]:
from sklearn.preprocessing import StandardScaler
# Other options are StandardScaler... others are available too!

# Prepare an example dataset:
correlated_part = np.random.normal(0,3,(1000))
x0 = np.random.normal(10,1,(1000)) + correlated_part
x1 = np.random.normal(10,2,(1000)) + correlated_part

# Make the ntuple
x_train = np.swapaxes(np.array((x0,x1)),0,1)
# x_train.shape

scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)

# Can also do x_scaled = scaler.fit(x_train).transform(x_train)
# or even x_scaled = scaler.fit_transform(x_train)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16, 6))
ax1.scatter(x0,x1)
ax2.scatter(x_train_scaled[:,0],x_train_scaled[:,1])
ax1.set_title('original')
ax2.set_title('scaled');

Principle Component Analysis (PCA)
==========

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(x_train_scaled)
x_pca = pca.transform(x_train_scaled)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16, 6))
ax1.scatter(x0,x1)
ax2.scatter(x_pca[:,0],x_pca[:,1])
ax1.set_title('original')
ax2.set_title('PCA');

In [ ]:
# The rotational components
pca.components_